### Loading base LM and our 18min-cooked ReFT model from HuggingFace

In [1]:
import torch, transformers
from pyreft import (
    ReftModel,
    get_intervention_locations
)

prompt_no_input_template = """Below is an instruction that \
describes a task. Write a response that appropriately \
completes the request.

### Instruction:
%s

### Response:
"""

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name_or_path = "meta-llama/Llama-2-7b-hf"
reft_model_name_or_path = "zhengxuanzenwu/Loreft1k-Llama-2-7b-hf"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048, padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)
reft_model = ReftModel.load(
    reft_model_name_or_path, model, from_huggingface_hub=True)
reft_model.set_device(device)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

### Inference with intervened locations

In [40]:
instruction = "Can you give any tips on how to cook a juicy, medium-rare steak?"

# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)
intervention_locations = torch.tensor([get_intervention_locations(
    last_position=prompt["input_ids"].shape[-1], positions="f5+l5",
    num_interventions=len(reft_model.interventions))]).permute(1, 0, 2).tolist()

# generate
_, reft_response = reft_model.generate(
    prompt, 
    unit_locations={"sources->base": (None, intervention_locations)},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=False, 
    no_repeat_ngram_size=5, repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

Keyword arguments {'add_special_tokens': False} not recognized.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Can you give any tips on how to cook a juicy, medium-rare steak?

### Response:
Sure! Here are some tips for cooking a juicy, delicious medium-rare steaks:
1. Choose high-quality beef with a good marbling of fat. This will ensure that your steak stays moist and tender while cooking.
2. Season your steak generously with salt and pepper before grilling or pan-frying it. This will help enhance its flavor.
3. Cook your steak over medium heat until it reaches an internal temperature of 140°F (60°C). This will ensure that the meat remains pink inside without overcooking it.
4. Let your steak rest for at least five minutes after removing it from the heat source so that all of the juices can redistribute throughout the meat.
5. Serve your steak with a side dish like mashed potatoes or sautéed vegetables for added flavor and texture contrast.

I hope these tips help you cr

### Llama-2 7B base LM can follow certain instructions already!

We follow previous works to evaluate with Llama-2; but you should try on other base LMs as well!

In [41]:
# generate
model_response = model.generate(
    **prompt, 
    max_new_tokens=512, do_sample=False, 
    no_repeat_ngram_size=5, repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(model_response[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Can you give any tips on how to cook a juicy, medium-rare steak?

### Response:
I've been cooking steaks for years and I have found that the best way to get a juicy, rare steak is to use a cast iron skillet. The cast iron skillet will help to sear the outside of the steak while keeping it moist on the inside. You should also make sure to season your cast iron skillet before using it so that it doesn't stick to the meat when cooking.

